In [ ]:
#This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [3]:
import keras.layers as L
import keras.models as M
import tensorflow as tf
from keras.utils import Sequence
import pandas as pd
import numpy as np

In [2]:
class Data(Sequence):
    def __init__(self,q_name,r_name,l_name=None,batch_size=50):
        self.r_batch_size = batch_size
        self.q_batch_size = int(batch_size/10)
        self.qdf = pd.read_csv(q_name,header=None,sep=',')
        self.rdf = pd.read_csv(r_name,header=None,sep=',')
        self.labels = pd.read_csv(l_name,header=None,sep=',')
        
    def __len__(self):
        return int(np.ceil(len(self.rdf) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_q = self.qdf[idx * self.q_batch_size:(idx + 1) * self.q_batch_size]
        batch_r = self.rdf[idx * self.r_batch_size:(idx + 1) * self.r_batch_size]
        batch_y = self.labels[idx * self.r_batch_size:(idx + 1) * self.r_batch_size]
        ratio = self.r_batch_size / self.q_batch_size
        batch_q = np.array([np.array([q]*ratio) for q in batch_q])
        
        return [batch_q, batch_r],batch_y

In [ ]:
train_seq = Data("Query_embeddings.csv","Train_resp_embeddings.csv","Train_labels.csv",50)
test_seq = Data("Query_test_embeddings.csv","Test_resp_embeddings.csv")

In [17]:
def model_create(q_inp_size, r_inp_size):
    inp1 = tf.keras.layers.Input(shape=q_inp_size)
    inp2 = tf.keras.layers.Input(shape=r_inp_size)
    
    f1 = tf.keras.layers.Add()([inp1,inp2])
    f2 = tf.keras.layers.Dot(axes=1)([inp1,inp2])
    f3 = tf.keras.layers.Multiply()([inp1,inp2])
    f4 = tf.keras.layers.Subtract()([inp1,inp2])
    
    comb = tf.keras.layers.Concatenate()([inp1,inp2,f1,f2,f3,f4])
    
    x = tf.keras.layers.Dense(1024,activation='relu')(comb)
    x = tf.keras.layers.Dense(512,activation='relu')(x)
    x = tf.keras.layers.Dense(256,activation='relu')(x)
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    out = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    
    model = tf.keras.models.Model(inputs=[inp1,inp2],outputs=[out])
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [18]:
model = model_create((30,),(30,))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
add_7 (Add)                     (None, 30)           0           input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
dot_5 (Dot)                     (None, 1)            0           input_15[0][0]                   
          

In [ ]:
model.fit_generator(train_seq,epochs=10,validation_data=test_seq)